# Выполнил Сивцов Артём.

Проект выполнен в Google Colab ноутбуке. Особых рекомендаций для запуска нет.

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!unzip /content/gdrive/MyDrive/CV/images.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: images/92f04ea993da153b5c7c88ac48814510.jpg  
  inflating: images/ab416b6b733e1a9acd4a7a8802642986.jpg  
  inflating: images/2089a58ae4f9e2247d99cc7b11d6b5da.jpg  
  inflating: images/74256f41648d49161b3279d5477fe1dc.jpg  
  inflating: images/c6f16416e9ac3cef445c76803804a3c1.jpg  
  inflating: images/06a598784129a6fbe204237314a8ecd3.jpg  
  inflating: images/378205fde419faa32f696fb436400007.jpg  
  inflating: images/f356a3be5d7cac22955ff7ede753f7cf.jpg  
  inflating: images/7b20b3f389d5f1cd8989c70ff312281a.jpg  
  inflating: images/8d4282db6903385e69a6bf1e493a78ed.jpg  
  inflating: images/bcca07e3ea7ca93e0a9401390c656e40.jpg  
  inflating: images/40b6650e3585d78bc74f7719b7d5fbbf.jpg  
  inflating: images/ef7350026636458f3cc6310a08849e41.jpg  
  inflating: images/39845aea7887fec2a1304bbcf3af35d3.jpg  
  inflating: images/5b56fa6cf618b0803ae05628f9f03c06.jpg  
  inflating: images/241e6ce8d94bdb8a0c8c73b5da7

Импортируем библиотеки:

In [73]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, AvgPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [74]:
df_g = pd.read_csv('/content/gdrive/MyDrive/CV/georges.csv', header=None)
df_g.columns = ['path']
df_g.head()

,path
0,https://i.pinimg.com/736x/17/0d/5b/170d5b93d80...
1,https://i.pinimg.com/736x/47/b9/9a/47b99a2ddcd...
2,https://i.pinimg.com/736x/90/e8/90/90e890f054b...
3,https://i.pinimg.com/736x/0a/71/6f/0a716f6f14e...
4,https://i.pinimg.com/736x/f1/95/be/f195bea0b78...


In [75]:
df_g.duplicated().sum()
df_g = df_g.drop_duplicates()
df_g.duplicated().sum()

0

In [76]:
df_g.shape

(2360, 1)

In [77]:
df_n = pd.read_csv('/content/gdrive/MyDrive/CV/non_georges.csv', header=None)
df_n.columns = ['path']
df_n.head()

,path
0,https://i.pinimg.com/736x/fd/65/bb/fd65bb679b7...
1,https://i.pinimg.com/736x/78/63/a2/7863a2404af...
2,https://i.pinimg.com/736x/bc/3a/f4/bc3af41444c...
3,https://i.pinimg.com/736x/46/a3/59/46a3595eec7...
4,https://i.pinimg.com/736x/54/b2/ef/54b2ef7af71...


In [78]:
df_n.duplicated().sum()
df_n = df_n.drop_duplicates()
df_n.duplicated().sum()

0

In [79]:
df_n.shape

(3340, 1)

## Загрузим данные с помощью wget

**Примечание: данные сохранены локально с помощью wget, а затем загружены на Гугл диск для дальнейшей работы на Google Colab.**

Следующими двумя командами сохраним списки адресов в тхт файл:

df_g['path'].to_csv('./images_g/g.txt', header=False, index=False, sep=',')

df_n['path'].to_csv('./images_n/n.txt', header=False, index=False, sep=',')

Затем с помощью команды cd (directory_name) перейдем в нужную директорию и с помощью команды wget -i (file_name).txt загрузим изображения из файла.

Затем загрузим архив с картинками на Google Drive и продолжим работать в ноутбуке Google Colab

In [80]:
from os.path import basename

df_g['file_name'] = df_g['path'].apply(basename)
df_g['label'] = 1
df_g['label_name'] = 'george'
df_g.head()

,path,file_name,label,label_name
0,https://i.pinimg.com/736x/17/0d/5b/170d5b93d80...,170d5b93d80d247be60f22ca1216bef7.jpg,1,george
1,https://i.pinimg.com/736x/47/b9/9a/47b99a2ddcd...,47b99a2ddcde2cf9e62b4faeec2fdf20.jpg,1,george
2,https://i.pinimg.com/736x/90/e8/90/90e890f054b...,90e890f054b920308987c140125c9d19.jpg,1,george
3,https://i.pinimg.com/736x/0a/71/6f/0a716f6f14e...,0a716f6f14eea3adc30e0944a7123759.jpg,1,george
4,https://i.pinimg.com/736x/f1/95/be/f195bea0b78...,f195bea0b78d1c0c90ab54341bf39536.jpg,1,george


In [81]:
df_n['file_name'] = df_n['path'].apply(basename)
df_n['label_name'] = 'non_george'
df_n['label'] = 0

df_n.head()

,path,file_name,label_name,label
0,https://i.pinimg.com/736x/fd/65/bb/fd65bb679b7...,fd65bb679b7975ab73867d62fba2e991.jpg,non_george,0
1,https://i.pinimg.com/736x/78/63/a2/7863a2404af...,7863a2404af5741f6d9fdcb2e4ffad02.jpg,non_george,0
2,https://i.pinimg.com/736x/bc/3a/f4/bc3af41444c...,bc3af41444c7d8e4403691eb324941fb.jpg,non_george,0
3,https://i.pinimg.com/736x/46/a3/59/46a3595eec7...,46a3595eec7e6df1088b1b8139672423.jpg,non_george,0
4,https://i.pinimg.com/736x/54/b2/ef/54b2ef7af71...,54b2ef7af71a9e522f6013b599eb31de.jpg,non_george,0


Объединим датафреймы с метками двух классов и названиями картинок в один датафрейм:

In [82]:
df = pd.concat([df_g, df_n], axis=0)
df.shape

(5700, 4)

Проверим наличие дубликатов:

In [83]:
df.duplicated().sum()

0

In [84]:
df['label'].value_counts()

0    3340
1    2360
Name: label, dtype: int64

In [85]:
3340 / 5700

0.5859649122807018

Создадим переменные с соотношениями классов:

In [86]:
geo = 5700 / 2681
non_geo = 5700 / 3340

In [87]:
df['y'] = df['label'].apply(lambda x: [0,1] if x == 1 else [1, 0])

In [88]:
df.head()

,path,file_name,label,label_name,y
0,https://i.pinimg.com/736x/17/0d/5b/170d5b93d80...,170d5b93d80d247be60f22ca1216bef7.jpg,1,george,"[0, 1]"
1,https://i.pinimg.com/736x/47/b9/9a/47b99a2ddcd...,47b99a2ddcde2cf9e62b4faeec2fdf20.jpg,1,george,"[0, 1]"
2,https://i.pinimg.com/736x/90/e8/90/90e890f054b...,90e890f054b920308987c140125c9d19.jpg,1,george,"[0, 1]"
3,https://i.pinimg.com/736x/0a/71/6f/0a716f6f14e...,0a716f6f14eea3adc30e0944a7123759.jpg,1,george,"[0, 1]"
4,https://i.pinimg.com/736x/f1/95/be/f195bea0b78...,f195bea0b78d1c0c90ab54341bf39536.jpg,1,george,"[0, 1]"


In [89]:
df['y'].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1, 0]    3340
[0, 1]    2360
Name: y, dtype: int64

Перемешаем итоговую таблицу по строкам:

In [90]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,path,file_name,label,label_name,y
0,https://i.pinimg.com/736x/38/cd/f2/38cdf2ebeb7...,38cdf2ebeb7ccd16e4de5234c6b4d463.jpg,0,non_george,"[1, 0]"
1,https://i.pinimg.com/736x/03/70/4e/03704ea2243...,03704ea22436ad2b35d1869ca5e2b014.jpg,1,george,"[0, 1]"
2,https://i.pinimg.com/736x/3d/52/bf/3d52bfeb626...,3d52bfeb626a0679588d3632f8e313ee.jpg,1,george,"[0, 1]"
3,https://i.pinimg.com/736x/97/04/7f/97047f7fc4f...,97047f7fc4fb5f84515e53cfc78f9a15.jpg,1,george,"[0, 1]"
4,https://i.pinimg.com/736x/cc/f8/76/ccf8763cb1f...,ccf8763cb1fd89740610552ee497e1ec.jpg,0,non_george,"[1, 0]"


С помощью загрузчика данных загрузим тренировочный и тестовый набор картинок:

In [91]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1, horizontal_flip=True)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='./images',
        x_col='file_name',
        y_col='label_name',
        target_size=(250, 250),
        batch_size=64,
        #class_mode='binary',
        subset='training',
        seed=42)

Found 5130 validated image filenames belonging to 2 classes.


In [92]:
test_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='./images',
        x_col='file_name',
        y_col='label_name',
        target_size=(250, 250),
        batch_size=64,
        #class_mode='binary',
        subset='validation',
        seed=42)

Found 570 validated image filenames belonging to 2 classes.


Также создадим лишний загрузчик с батчем равным количеству объектов для оценки метрик после обучения модели.

In [93]:
test_gen_flow_full = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='./images',
        x_col='file_name',
        y_col='label_name',
        target_size=(250, 250),
        batch_size=570,
        #class_mode='binary',
        subset='validation',
        seed=42)

Found 570 validated image filenames belonging to 2 classes.


In [94]:
_, target = next(test_gen_flow_full)
target.shape

(570, 2)

Исследуем несколько архитектур нейронных сетей на нашей задаче:

In [95]:
# RESNET50+ADAM
tf.keras.backend.clear_session()
backbone = ResNet50(weights='imagenet',
                        input_shape=(250, 250, 3),
                        include_top=False)



model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))

model.add(Dense(2048, activation='relu'))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(lr=1e-4)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 27,788,162
Trainable params: 27,735,042
Non-trainable params: 53,120
_________________________________________________________________


In [96]:
#RESNET50+SGD
tf.keras.backend.clear_session()
backbone = ResNet50(weights='imagenet',
                        input_shape=(250, 250, 3),
                        include_top=False)

model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())

model.add(Dense(2048, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 27,788,162
Trainable params: 27,735,042
Non-trainable params: 53,120
_________________________________________________________________


In [97]:
#LENET+SGD
tf.keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same',
                 activation="tanh", input_shape=(250, 250, 3)))
 
model.add(AvgPool2D(pool_size=(2, 2),padding='valid', strides=None))
 
model.add(Conv2D(filters=16, kernel_size=(5, 5), padding='valid',
                 activation="tanh", input_shape=(14, 14, 6)))
 
model.add(AvgPool2D(pool_size=(2, 2),padding='valid', strides=None))
 
model.add(Flatten())

model.add(Dense(units=120, activation='tanh'))
model.add(Dense(units=84, activation='tanh'))
model.add(Dense(units=10, activation='softmax'))

model.add(Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 125, 125, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 16)      2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 60, 60, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 120)               6912120   
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [98]:
# ALEXNET+ADAM
from tensorflow import keras 
tf.keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(250,250,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
])
optimizer = Adam(lr=0.0001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 60, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 384)       8

В выводе оставлен результат работы последней нейронки - AlexNet, т.к. с её помощью получился наилучший результат

In [99]:
%%time
model.fit(train_gen_flow, epochs=10, verbose=1, class_weight={1:geo, 0:non_geo}, 
          validation_data=test_gen_flow)

Epoch 1/10
81/81 [==============================] - 64s 778ms/step - loss: 4.5681 - accuracy: 0.5978 - val_loss: 0.9990 - val_accuracy: 0.6263
Epoch 2/10
81/81 [==============================] - 62s 766ms/step - loss: 1.5645 - accuracy: 0.6893 - val_loss: 0.6825 - val_accuracy: 0.6263
Epoch 3/10
81/81 [==============================] - 62s 767ms/step - loss: 1.4212 - accuracy: 0.7013 - val_loss: 0.6704 - val_accuracy: 0.6298
Epoch 4/10
81/81 [==============================] - 62s 768ms/step - loss: 1.2219 - accuracy: 0.7312 - val_loss: 0.7274 - val_accuracy: 0.4947
Epoch 5/10
81/81 [==============================] - 62s 766ms/step - loss: 1.1006 - accuracy: 0.7387 - val_loss: 0.6280 - val_accuracy: 0.6351
Epoch 6/10
81/81 [==============================] - 62s 767ms/step - loss: 1.0447 - accuracy: 0.7483 - val_loss: 0.5770 - val_accuracy: 0.6807
Epoch 7/10
81/81 [==============================] - 62s 769ms/step - loss: 0.9539 - accuracy: 0.7663 - val_loss: 0.4923 - val_accuracy: 0.7807

In [100]:
predicted = model.predict(test_gen_flow, verbose=1)

9/9 [==============================] - 6s 663ms/step


In [101]:
predicted

array([[0.35347712, 0.6465229 ],
       [0.8735227 , 0.12647724],
       [0.98656464, 0.01343535],
       ...,
       [0.3152663 , 0.6847337 ],
       [0.03331235, 0.9666876 ],
       [0.42046198, 0.57953805]], dtype=float32)

In [102]:
predicted = np.argmax(predicted, axis=1)
np.unique(predicted)

array([0, 1])

In [103]:
f1_score(np.argmax(target, axis=1), predicted)

0.6927939317319849

In [104]:
accuracy_score(np.argmax(target, axis=1), predicted)

0.5736842105263158

Получена точность 0.57 и ф-мера 0.69